# Exploratory Data Analysis of Heavy Truck J1939 data

Ce notebook va servir de base dans l'analyse des données du `Heavy Truck`. Ces données reprennent une payload que nous pouvons retrouver avec le protocole **bus CAN J1939**.

**Payload d'un message sous protocole J1939:**

![payload j1939](./images/j1939-payload.png)

:warning: Attention à bien écouter Vendredi 26 la présentation de `Simon Bellemare` sur les données **bus CAN J1939** !

## Imports

In [1]:
from sqlmodel import Session, select

from canlock.db.database import get_session, init_db
from canlock.db.models import Vehicle, CanMessage, PgnDefinition, SpnDefinition, Ecu, Session

1ère étape: récupérons toutes les sessions

In [2]:
init_db()

with get_session() as session:
    sessions = session.exec(select(Session)).all()
    print(f"Number of sessions: {len(sessions)}")

Number of sessions: 228


2ème étape: sélectionnons une session

In [3]:
session_durations = [(i, (sess.end_time - sess.start_time).total_seconds() / 60) for i, sess in enumerate(sessions)]
[(sess_id, duration) for sess_id, duration in session_durations if duration <= 10 and duration > 0]

[(9, 0.9833333333333333),
 (22, 9.866666666666667),
 (25, 0.9833333333333333),
 (46, 1.0),
 (55, 0.9833333333333333),
 (57, 2.95),
 (63, 1.9666666666666666),
 (68, 0.9833333333333333),
 (72, 3.933333333333333),
 (73, 3.933333333333333),
 (75, 1.9833333333333334),
 (81, 3.933333333333333),
 (89, 8.85),
 (99, 2.933333333333333),
 (102, 3.95),
 (119, 1.9666666666666666),
 (128, 8.866666666666667),
 (130, 0.9833333333333333),
 (152, 2.95),
 (161, 1.9666666666666666),
 (179, 2.966666666666667),
 (186, 8.883333333333333),
 (202, 1.9666666666666666),
 (205, 6.9),
 (206, 2.95),
 (208, 8.85),
 (214, 8.9)]

Prenons la session 205 qui a une durée d'environ 7 minutes, ce qui est largement suffisant pour nos tests.

In [4]:
with get_session() as session:
    can_messages = session.exec(
        select(CanMessage)
        .where(CanMessage.session_id == sessions[205].id)
        .order_by(CanMessage.timestamp)
    ).all()
    
print(f"Nombre de messages CAN disponible durant cette session: {len(can_messages)}")
can_messages[:5]

Nombre de messages CAN disponible durant cette session: 400008


[CanMessage(can_identifier=369055206, length=8, id=424108483, timestamp=datetime.datetime(2021, 3, 18, 12, 4, 15, 211130), session_id=UUID('c6c2d871-9725-4b8d-8810-0b0b844f9d09'), payload=b'\xbf\x7f\xff\xbc\xff\x03<\x00'),
 CanMessage(can_identifier=418384358, length=8, id=424108484, timestamp=datetime.datetime(2021, 3, 18, 12, 4, 15, 211696), session_id=UUID('c6c2d871-9725-4b8d-8810-0b0b844f9d09'), payload=b'Y}`{}\xc7|\xff'),
 CanMessage(can_identifier=419323080, length=8, id=424108485, timestamp=datetime.datetime(2021, 3, 18, 12, 4, 15, 212277), session_id=UUID('c6c2d871-9725-4b8d-8810-0b0b844f9d09'), payload=b'\xff\xff\xff\xff\xff\x00\xff\xff'),
 CanMessage(can_identifier=419361254, length=8, id=424108486, timestamp=datetime.datetime(2021, 3, 18, 12, 4, 15, 212782), session_id=UUID('c6c2d871-9725-4b8d-8810-0b0b844f9d09'), payload=b'\xf7\x00\x00\xc4\x00\xff\xe0\xff'),
 CanMessage(can_identifier=150892262, length=8, id=424108487, timestamp=datetime.datetime(2021, 3, 18, 12, 4, 15, 215

Cela nous fait environ 400000 messages à décoder, ce qui est amplement suffisant.

Pour récupérer les valeurs, il faut dans un premier temps, identifier le bon PGN. Puis avec le PGN, nous devons récupérer les SPNs qui sont associés et décoder les valeurs par rapports aux colonnes données.

In [ ]:
init_db()  # Ensure tables are created

with get_session() as session:
    # Fetch all vehicles
    vehicles = session.exec(select(Vehicle)).all()
    
    # Fetch 100 CAN messages for example
    canmessages = session.exec(select(CanMessage).limit(100)).all()

In [ ]:
vehicles

In [ ]:
canmessages

In [ ]:
canmessages[0].can_identifier

# Convert integer to 29 bits string
binary_identifier = bin(canmessages[0].can_identifier)[2:].zfill(29)
pgn_identifier = binary_identifier[3:21] # Extract bits 3 to 20

# Convert PGN identifier to integer
pgn_integer = int(pgn_identifier, 2)

# SQLmodel query to table PGNDefinition with pgn_integer key value
with get_session() as session:
    pgn_definition = session.exec(select(PgnDefinition).where(PgnDefinition.pgn_identifier == pgn_integer)).first()

In [ ]:
pgn_identifier

In [ ]:
pgn_integer

In [ ]:
pgn_definition.name

In [ ]:
canmessages[0].can_identifier

# Convert integer to 29 bits string
binary_identifier = bin(canmessages[0].can_identifier)[2:].zfill(29)
address_identifier = binary_identifier[21:] # Extract bits 21 to the end

# Convert Address identifier to integer
address_integer = int(address_identifier, 2)

# SQLmodel query to table Ecu with pgn_integer key value
with get_session() as session:
    ecu_result = session.exec(select(Ecu).where(Ecu.address == address_integer)).first()

In [ ]:
ecu_result.name

In [ ]:
with get_session() as session:
    all_spn_definitions = session.exec(select(SpnDefinition)).all()
    all_pgn_definitions = session.exec(select(PgnDefinition)).all()

In [ ]:
len(all_spn_definitions)

In [ ]:
len(all_pgn_definitions)

In [ ]:
all_spn_names = [spn_def.name for spn_def in all_spn_definitions]
all_pgn_names = [pgn_def.name for pgn_def in all_pgn_definitions]

In [ ]:
set(all_spn_names).intersection(set(all_pgn_names))

In [ ]:
all_spn_definitions

In [ ]:
all_pgn_definitions

In [ ]:
# Convert payload to 64-bits binary string
binary_payload = bin(int.from_bytes(canmessages[7].payload, byteorder='big'))[2:].zfill(64)
binary_payload

In [ ]:
binary_payload[4:12]

In [ ]:
binary_payload[24:40]

In [ ]:
int(binary_payload[24:40], 2)

In [ ]:
with get_session() as session:
    spn_definition = session.exec(select(SpnDefinition).where(SpnDefinition.spn_identifier == int(binary_payload[24:40], 2))).first()
    analog_definition = spn_definition.analog_attributes

In [ ]:
analog_definition

In [ ]:
binary_payload[8:16]